# Data Preparation and Augmentation

Data augmentation techniques include RandomResizedCrop, RandomHorizontalFlip, RandAugment (applies random augmentations), and RandomErasing (randomly erases a rectangular region in an image). The pixel values are also normalized using the mean and standard deviation of the dataset. I found this in my ResNet-34 from scratch project.

Dataset is then split into a 90% training set and a 10% validation set.

In [31]:
import torch
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from torchvision.transforms import RandAugment
from torch import nn, optim
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

image_size = 64

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(image_size, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    RandAugment(num_ops=2, magnitude=9),
    transforms.ToTensor(),
    transforms.Normalize(mean=([0.4802, 0.4481, 0.3975]),   # normalize using mean & std
                         std=([0.2296, 0.2263, 0.2255])),
    transforms.RandomErasing(p=0.25, scale=(0.02, 0.33), ratio=(0.3, 3.3), value='random'),
])

val_transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=([0.4802, 0.4481, 0.3975]),   # normalize using mean & std
                         std=([0.2296, 0.2263, 0.2255])),
])

data_dir = 'tiny-imagenet-200/train'
full_dataset = datasets.ImageFolder(root=data_dir, transform=train_transform)

train_size = int(0.9 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

val_dataset.dataset.transform = val_transform

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=4, pin_memory=True)


Using device: cuda


In [32]:
img, label = train_dataset[0]
img.shape

torch.Size([3, 64, 64])

## Patching and Splitting Windows
Unlike traditional CNNs that use sliding convolutional filters, Vision Transformers break the image down into a sequence of patches, treating them similarly to words in a sentence.

Patchify Class: Takes an input image and converts it into patch embeddings. It uses a single convolutional layer where the kernel size and stride are equal to the patch_size. It divides the image into non overlapping patches and creating an initial vector embedding for each one.

split_into_windows: Takes the patches and splits them into smaller windows. Self-attention is calculated within these windows, which is far more computationally efficient than the original ViT's approach of global attention across all patches.

reverse_windows: Merges the split windows back into their original spatial layout.

In [33]:
class Patchify(nn.Module):
    """
    Convert an image into patch embeddings using a convolutional layer.

    Args:
        in_channels: Number of input channels (e.g., 3 for RGB).
        embed_dim: Output embedding dimension per patch.
        patch_size: Size of each square patch

    Returns:
        Tensor of shape (BS, H//patch_size, W//patch_size, embed_dim)
    """
    def __init__(self, in_channels, embed_dim, patch_size):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, embed_dim, kernel_size=patch_size, stride=patch_size)
    
    def forward(self, x):
        # x shape = (B, 3, H, W)
        x = self.conv(x)         # (BS, embed_dim, H//patch_size, W//patch_size)
        return x.permute(0, 2, 3, 1) # (BS, H//patch_size, W//patch_size, embed_dim)


In [34]:
def split_into_windows(x, M):
    """
    Splits (BS, H, W, channels) into non overlapping MxM windows.
    Args:
        x: Tensor of shape (BS, H, W, channels)
        M: Window size

    Returns:
        Tensor of shape (BS * num_windows, M*M, channels)
    """
    BS, H, W, channels = x.shape
    x = x.reshape(BS, H//M, M, W//M, M, channels)
    # Permute fixes the order so the MxM pixels are together properly
    x = x.permute(0, 1, 3, 2, 4, 5)     # (BS, H//M, W//M, M, M, channels)
    return x.reshape(-1, M*M, channels) # (BS * num_windows, M*M, channels)


In [35]:
def reverse_windows(x, M, H, W, channels):
    """
    Reverses MxM window tokens back into the original layout.

    Args:
        x: Tensor of shape (BS * num_windows, M*M, channels)
        M: Window size
        H: Original image height
        W: Original image width
        channels: Number of channels

    Returns:
        Tensor of shape (BS, H, W, channels)
    """
    BS = x.shape[0] // (H//M * W//M) # Original BS
    x = x.reshape(BS, H//M, W//M, M, M, channels)
    x = x.permute(0, 1, 3, 2, 4, 5)      # (BS, H//M, M, W//M, M, channels)
    return x.reshape(BS, H, W, channels) # (BS, H, W, channels)

## Relative Position Bias

Relative Position Bias shows self-attention mechanism about the geometry of the image. It computes a learnable bias between every pair of tokens in a window, based on how far apart they are. For each pair of tokens in an M x M window, compute relative position, use that to index into a learnable bias table, add this bias to the attention logits.

In [36]:
class RelativePositionBias(nn.Module):
    """
    Computes relative position bias for self-attention.

    Generates a table of learnable relative position biases between token pairs
    within an attention window of shape (M x M). The relative position between
    any two tokens is encoded as a bias vector per attention head, and these biases are
    added to the attention scores in self-attention.

    Args:
        M: The height/width of the attention window. The total number of tokens is M * M.
        nheads: Number of attention heads.

    Attributes:
        relative_table: Learnable parameter of shape ((2M - 1)^2, nheads),
            where each entry represents a bias value for a specific relative position and head.
        relative_index: Lookup table of shape (M*M, M*M), where each entry is an index
            into relative_table that maps the relative position between two tokens to a bias vector.

    Forward Output:
        Tensor of shape (nheads, M*M, M*M) containing the relative bias for each token pair
        and each attention head. This can be directly added to attention logits.

    Example:
        relative = RelativePositionBias(M=3, nheads=4)
        bias = relative()  # Output shape: (4, 9, 9), for 4 heads and 3x3 tokens
    """
    def __init__(self, M, nheads):
        super().__init__()
        self.M = M
        self.nheads = nheads
        # (2M-1)^2 because there are up to M-1 tokens above or below or left or right of each token.
        # 2M - 1 possibilities for above and below, same for left and right. So (2M -1)^2 total.
        # If M = 3 row and col would go between -2 and +2 when comparing two tokens.
        # That gives 5^2 possible combinations. len(-2, -1, 0, 1, 2)^2
        self.relative_table = nn.Parameter(torch.zeros(size=((2*M - 1) * (2*M -1), nheads)))

        # Coordinate grid of token positions shows where each token is in the window.
        # It gives every token a (row, col) coordinate.
        # If M = 3: coords[0] (rows): [[0, 0, 0], [1, 1, 1], [2, 2, 2]], 
        #           coords[1] (cols): [[0, 1, 2], [0, 1, 2], [0, 1, 2]]
        coords = torch.stack(torch.meshgrid( # Matrix style over Cartesian
            torch.arange(M), torch.arange(M), indexing='ij'))  # (2, M, M)
        
        # Flatten the coordinates for token indices, so coords[:, i] is the (row, col) of token i
        coords = coords.flatten(1) # (2, M*M)

        # Compute relative positions so we have the position of a token relative to another token
        # coords[:, :, None] shape: (2, M*M, 1), coords[:, None, :] shape: (2, 1, M*M)
        relative = coords[:, :, None] - coords[:, None, :] # (2, M*M, M*M)

        # Reformat so we can use each (row, col) as an index into a table but row/col values
        # range from -(M-1) to (M-1) so we shift them up so they are positive: [0, 2M -2]
        relative = relative.permute(1, 2, 0) # (M*M, M*M, 2)
        relative[:, :, 0] += M - 1
        relative[:, :, 1] += M - 1

        # Flatten 2D positions into 1D. To convert: row * num_cols + col
        self.register_buffer(   # Register buffer to move to GPU
            "relative_index",
            (relative[:, :, 0] * (2*M - 1) + relative[:, :, 1]).long() # (M*M, M*M)
        )
    def forward(self): 
        # Use index to get bias values, look up the bias vector for each token pair
        bias = self.relative_table[self.relative_index.view(-1)] # (M*M * M*M, nheads)
        bias = bias.reshape(self.relative_index.shape[0], self.relative_index.shape[1], self.nheads)
        return bias.permute(2, 0, 1) # (nheads, M*M, M*M)


## Windowed Multi-Head Self-Attention (W-MSA)

Implements Windowed Multi-head Self-Attention, which is a more efficient version of the standard attention used in ViT. Instead of calculating attention across all patches in the entire image, W-MSA computes attention within M x M windows. Significantly reduces the number of calculations needed.

Attention(Q, K, V) = Softmax((QK.T / √d) + B + mask) @ V

In [37]:
class WindowAttention(nn.Module):
    """
    Window-based Multihead Self-Attention with relative position bias.

    Performs self-attention within non overlapping MxM windows of the input feature map.
    It incorporates relative positional encoding and attention masks for shifted windows

    Attention(Q, K, V) = Softmax((QK.T / √d) + B + mask) @ V
    B is relative position bias.
    Args:
        channels: Input channels
        M: Height and width of the attention window.
        nheads: Number of attention heads.

    Attributes:
        q, k, v: Linear layers for queries, keys, and values.
        out: Output linear layer after attention.
    """
    def __init__(self, channels, M, nheads):
        super().__init__()
        self.M = M
        self.nheads = nheads
        self.rootd = (channels // nheads) ** -0.5 # (1 / √d) == (1 / √dim_per_head)

        self.q = nn.Linear(channels, channels)
        self.k = nn.Linear(channels, channels)
        self.v = nn.Linear(channels, channels)
        self.out = nn.Linear(channels, channels)

        self.relative = RelativePositionBias(M, nheads)
    
    def forward(self, x, attn_mask=None):
        """
        Forward pass for window based self-attention.

        Args:
            x: shape (B * nW, M*M, channels) where nW is number of windows
            attn_mask: Attention mask used for shifted windows to prevent cross-window attention.

        Returns:
            Tensor of shape (B*nW, M*M, channels)
        """
        # x shape = (B * nW, M*M, channels) where nW is number of windows
        BnW, M_sq, channels = x.shape # M_sq is M*M

        # d stands for dim_per_head. Permute on k so no transpose when computing attn.
        # q: (BnW, M*M, channels) --> (BnW, M*M, nheads, d) --> transpose(1, 2) --> (BnW, heads, M*M, d)
        # k: (BnW, M*M, channels) --> (BnW, M*M, nheads, d) --> permute(0, 2, 3, 1) --> (BnW, heads, d, M*M)
        # v: (BnW, M*M, channels) --> (BnW, M*M, nheads, d) --> transpose(1, 2) --> (BnW, heads, M*M, d)
        q = self.q(x).reshape(BnW, M_sq, self.nheads, channels // self.nheads).transpose(1, 2)
        k = self.k(x).reshape(BnW, M_sq, self.nheads, channels // self.nheads).permute(0, 2, 3, 1)
        v = self.v(x).reshape(BnW, M_sq, self.nheads, channels // self.nheads).transpose(1, 2)

        # Attention: (Q @ K.T) / √d + relative bias + optional mask
        # k is already transposed.
        attn = (q @ k) * self.rootd # (BnW, nheads, M*M, M*M)
        attn = attn + self.relative()

        if attn_mask is not None:
            # attn_mask: (M*M, M*M) --> unsqueeze(0) --> (1, M*M, M*M)
            # Broadcasted to (BnW, nheads, M*M, M*M)
            nW = attn_mask.shape[0]
            BS = BnW // nW
            attn_mask = attn_mask.to(attn.device)
            attn = attn.view(BS, nW, self.nheads, M_sq, M_sq) + attn_mask.unsqueeze(0).unsqueeze(2)
            attn = attn.view(BnW, self.nheads, M_sq, M_sq)
        
        attn = attn.softmax(dim=-1)
        out = (attn @ v).transpose(1, 2).reshape(BnW, M_sq, channels)
        return self.out(out)
        

In [38]:
def create_attention_mask(H, W, M, shift):
    """
    Creates an attention mask for shifted window self-attention (SW-MSA).

    This function generates a mask to prevent tokens from attending across windows when 
    performing SW-MSA. It divides the feature map into distinct regions, assigns unique labels
    to each, uses cyclic shifting, partitions it into non overlapping windows, and then
    builds an attention mask that blocks attention between different labeled regions.

    Args:
        H: Height of the feature map.
        W: Width of the feature map.
        M: Window size.
        shift: Number of pixels to cyclically shift the window. 
               If shift is 0, no mask is needed. In Swin it is M // 2

    Returns:
        Tensor of shape (nW, M*M, M*M) where nW is the number of windows.
        Or None if shift is 0.
    """
    if shift == 0:
        return None
    
    img_mask = torch.zeros((1, H, W, 1))  # Mask

    count = 0
    H, W = img_mask.shape[1:3]

    # Split image into 9 regions
    h_ranges = [(0, H - M), (H - M, H - shift), (H - shift, H)]
    w_ranges = [(0, W - M), (W - M, W - shift), (W - shift, W)]

    # so if H = W = 12, M = 6, shift = 3
    # h_ranges = [(0, 6), (6, 9), (9, 12)]
    # w_ranges = [(0, 6), (6, 9), (9, 12)]

    # Fill each region with a unique integer
    for h_start, h_end in h_ranges:
        for w_start, w_end in w_ranges:
            img_mask[:, h_start:h_end, w_start:w_end, :] = count
            count += 1
            
    # Cyclic shift the mask
    img_mask = torch.roll(img_mask, shifts=(-shift, -shift), dims=(1,2))

    # Split into M*M windows
    mask_windows = split_into_windows(img_mask, M)  # (nW, M*M, 1)
    mask_windows = mask_windows.squeeze(-1)       # (nW, M*M)
    # Create attention mask
    attn_mask = mask_windows.unsqueeze(1) - mask_windows.unsqueeze(2)  # (nW, M*M, M*M)
    attn_mask = attn_mask.masked_fill(attn_mask != 0, float('-inf')).masked_fill(attn_mask == 0, 0.0)
    return attn_mask


In [39]:
create_attention_mask(12, 12, 6, 3)

tensor([[[0., 0., 0.,  ..., -inf, -inf, -inf],
         [0., 0., 0.,  ..., -inf, -inf, -inf],
         [0., 0., 0.,  ..., -inf, -inf, -inf],
         ...,
         [-inf, -inf, -inf,  ..., 0., 0., 0.],
         [-inf, -inf, -inf,  ..., 0., 0., 0.],
         [-inf, -inf, -inf,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., -inf, -inf, -inf],
         [0., 0., 0.,  ..., -inf, -inf, -inf],
         [0., 0., 0.,  ..., -inf, -inf, -inf],
         ...,
         [-inf, -inf, -inf,  ..., 0., 0., 0.],
         [-inf, -inf, -inf,  ..., 0., 0., 0.],
         [-inf, -inf, -inf,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., -inf, -inf, -inf],
         [0., 0., 0.,  ..., -inf, -inf, -inf],
         [0., 0., 0.,  ..., -inf, -inf, -inf],
         ...,
         [-inf, -inf, -inf,  ..., 0., 0., 0.],
         [-inf, -inf, -inf,  ..., 0., 0., 0.],
         [-inf, -inf, -inf,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., -inf, -inf, -inf],
         [0., 0., 0.,  ..., -inf, -inf, -inf],
         [0.

## Stochastic Depth

Stochastic depth is a regularization technique used to improve generalization and reduce overfitting. Instead of dropping individual neurons, entire residual branches are skipped during training with a given probability (drop_prob).

In [40]:
def stochastic_depth(x, drop_prob, training):
    """
    Applies stochastic depth to the input.

    Args:
        x: Input tensor
        drop_prob: Probability of dropping the path.
        training: If True, stochastic depth is applied. If False, input is returned unchanged.

    Returns:
        Output tensor with some residual paths zeroed out.
    """
    if drop_prob == 0.0 or not training:
        return x

    keep_prob = 1.0 - drop_prob
    # Create mask with shape (BS, 1, 1, ..., 1) so it broadcasts over all non batch dims
    shape = (x.shape[0],) + (1,) * (x.ndim - 1)
    # 1 with prob keep_prob, 0 with prob drop_prob
    mask = torch.rand(shape, dtype=x.dtype, device=x.device) < keep_prob
    # Scale
    x = x / keep_prob
    return x * mask

class StochasticDepth(nn.Module):
    """
    Module for stochastic depth.
    """
    def __init__(self, drop_prob):
        super().__init__()
        self.drop_prob = float(drop_prob)

    def forward(self, x):
        return stochastic_depth(x, self.drop_prob, self.training)


## SwinBlock (Shifted Window Transformer Block)

Applies windowed self-attention over non overlapping M×M windows and alternates between non-shifted and shifted windows across consecutive blocks to enable cross-window connections. There is also an MLP with GELU at the end. Residual connections are used.

![Block Architecture](figures/block_arch.png)

In [41]:
class SwinBlock(nn.Module):
    """
    Swin Transformer block with shifted/non-shifted window self-attention + MLP.

    Args:
        dim: Channel dimension of the input features.
        H: Feature map height.
        W: Feature map width.
        nheads: Number of attention heads in WindowAttention.
        M: Window size.
        shift: Cyclic shift size (0 for non-shifted windows, M//2 for shifted).
        ratio: Expansion ratio for the MLP hidden size (hidden dim = ratio * dim).
        stoch_depth: stochastic depth probability for dropping residual branches.

    Attributes:
        norm1: Pre attention normalization.
        attn (WindowAttention): Window-based multi-head self-attention.
        drop_path: Stochastic depth module or identity if stoch_depth == 0.
        norm2: Pre MLP normalization.
        mlp: Two-layer feed forward network with GELU activation.
        attn_mask: Mask for shifted attention, shape (nW, M*M, M*M) when shift > 0, else None.

    Input:
        x: Tensor of shape (BS, H, W, channels).

    Output:
        Tensor of shape (BS, H, W, Channels), same spatial shape and channels as input.

    """
    def __init__(self, dim, H, W, nheads, M, shift, ratio, stoch_depth):
        super().__init__()
        self.dim = dim
        self.M = M
        self.shift = shift
        self.norm1 = nn.LayerNorm(dim)
        self.attn = WindowAttention(channels=dim, M=M, nheads=nheads)
        if stoch_depth > 0:
            self.stoch = StochasticDepth(stoch_depth)
        else:
            self.stoch = nn.Identity()
        self.norm2 = nn.LayerNorm(dim)
        self.mlp = nn.Sequential(
            nn.Linear(dim, int(dim * ratio)), # Error I had before, forgot to wrap with int()
            nn.GELU(),
            nn.Linear(int(dim * ratio), dim)
        )
        self.attn_mask = create_attention_mask(H, W, M, shift)
    
    def forward(self, x):
        """
        Forward pass of SwinBlock.

        LayerNorm --> cyclic shift --> split windows --> WindowAttention (masked if shifted)
        --> reverse windows --> reverse shift --> residual + stochastic depth --> LayerNorm
        --> MLP → residual +stochastic depth

        Returns:
            Tensor of shape (BS, H, W, channels).
        """
        BS, H, W, channels = x.shape

        store = x # For Residual connection
        x = self.norm1(x)

        if self.shift > 0:
            x = torch.roll(x, shifts=(-self.shift, -self.shift), dims=(1, 2)) # Cyclic shift
        x_windows = split_into_windows(x, self.M)                  # (BS*nW, M*M, channels)
        x_windows = self.attn(x_windows, attn_mask=self.attn_mask) # (BS*nW, M*M, channels)
        x = reverse_windows(x_windows, self.M, H, W, channels)     # (BS, H, W, channels)
        if self.shift > 0:
            x = torch.roll(x, shifts=(self.shift, self.shift), dims=(1, 2))

        x = store + self.stoch(x)

        return x + self.stoch(self.mlp(self.norm2(x)))

## Patch Merging Layer

In [42]:
class PatchMerging(nn.Module):
    """
    Reduces H and W by 2x in each dimension. Doubles the channel dimension.
    - Extract non overlapping 2x2 patches from the feature map.
    - Concatenate features from each patch along the channel dimension.
    - Apply LayerNorm for normalization across channels.
    - Lower 4*channels down to 2*channels with a Linear layer.

    Input:
        x: shape (BS, H, W, channels)

    Output:
        shape (BS, H/2, W/2, 2*channels)

    H and W should be even.
    """
    def __init__(self, dim):
        super().__init__()
        self.lin = nn.Linear(4*dim, 2*dim)
        self.norm = nn.LayerNorm(4*dim) # 4 * channels --> 2 * channels
    
    def forward(self, x):
        # x shape = (BS, H, W, channels)
        x1 = x[:, 0::2, 0::2, :]
        x2 = x[:, 1::2, 0::2, :]
        x3 = x[:, 0::2, 1::2, :]
        x4 = x[:, 1::2, 1::2, :]
        x = torch.cat([x1, x2, x3, x4], dim=3) # (BS, H/2, W/2, 4*channels)
        x = self.norm(x)
        return self.lin(x) # (BS, H/2, W/2, 2*channels)


## Stage (Stack of Swin Blocks + Optional Patch Merging)

A Stage stacks SwinBlocks, alternating between:
- W-MSA (non-shifted windows, shift=0)
- SW-MSA (shifted windows, shift=M//2)

Then optionally applies PatchMerging to downsample and increase channels for the next stage.
After PatchMerging, the next stage should be constructed with dim = 2 * previous_dim and H and W halved.

In [43]:
class Stage(nn.Module):
    """
    Args:
        dim: Channel dimension.
        H: Feature map height.
        W: Feature map width.
        M: Window size.
        blocks: Number of SwinBlocks in this stage.
        nheads: Number of attention heads per block.
        stoch_depth_list: List of stochastic depth probabilities (len == blocks).
        patch_merging: If True, apply PatchMerging at the end of the stage.
        ratio: MLP expansion ratio (hidden dim = ratio * dim).

    Input:
        x: Tensor of shape (BS, H, W, channels or dim)

    Output:
        - If patch_merging is False: (BS, H, W, dim)
        - If patch_merging is True:  (BS, H/2, W/2, 2*dim)
    """
    def __init__(self, dim, H, W, M, blocks, nheads, stoch_depth_list, patch_merging, ratio):
        # stoch_depth_list is a list of the stochastic depth rates for each SwinBlock.

        super().__init__()
        self.blocks = nn.ModuleList()
        for i in range(blocks):
            if i % 2 == 0:
                shift = 0  # Alternate between W-MSA and SW-MSA, W-MSA has no shift.
            else:
                shift = M // 2
            self.blocks.append(
                SwinBlock(dim, H, W, nheads, M, shift, ratio, stoch_depth_list[i])
            )
        if patch_merging:
            self.patch = PatchMerging(dim)
        else:
            self.patch = nn.Identity()
    
    def forward(self, x):
        # x shape = (BS, H, W, channels)
        for block in self.blocks:
            x = block(x)
        return self.patch(x)

## Parameters

- Patch size: 4 × 4
- Base embed dim C: 96
- Depths: [2, 2, 6, 2]
- Num heads: [3, 6, 12, 24]
- Window size: 7 for all blocks
- Shift: 3 (7 // 2) for all shift blocks
- MLP expantion ratio: 4.0 for all blocks
- Drop path rate: 0.2 (linearly increased across all blocks)
- Patch Merging / Downsample at the end of Stage 1, 2 and 3.


| Stage         | Blocks | Heads | Stoch_dep | In Channels | Out Channels | Output Shape             |
|---------------|--------|-------|-----------|-------------|--------------|--------------------------|
| PatchEmbed    | None   | None  | None           | 3           | 96           | (BS, H/4, W/4, 96)       |
| Stage 1       | 2      | 3     | [0.0000, 0.0182]          | 96          | 192          | (BS, H/8, W/8, 192)      |
| Stage 2       | 2      | 6     | [0.0364, 0.0545]          | 192         | 384          | (BS, H/16, W/16, 384)    |
| Stage 3       | 6      | 12    | [0.0727, 0.0909, 0.1091, 0.1273, 0.1455, 0.1636]          | 384         | 768          | (BS, H/32, W/32, 768)    |
| Stage 4       | 2      | 24    | [0.1818, 0.2000]          | 768         | 768          | (BS, H/32, W/32, 768)    |
| Head          | None   | None  | None          | 768         | num_classes  | (BS, num_classes)        |

## Swin Transformer Architecture

![Architecture](figures/Architecture.png)

In [44]:
class SwinTransformer(nn.Module):
    """
    Swin Transformer

    Steps:
        - Patchify to get (BS, H/patch, W/patch, emb_dim)
        - 4 stages of Swin blocks with alternating W-MSA (shift=0) and SW-MSA (shift=M//2)
        - Patch Merging at the end of stages 1-3
        - Global average pooling and linear classifier head

    Args:
        img_size: Input image size
        patch_size: Patch size for Patchify.
        emb_dim: Base embedding dimension channel for stage 1.
        blocks (List[int]): Number of blocks per stage.
        nheads (List[int]): Number of attention heads per stage.
        M: Window size for all blocks.
        n_classes: Number of output classes for the classifier head.
        stochastic_endpoint: Stochastic depth ratio endpoint for linspace.

    Shapes:
        Input:  (BS, 3, 224, 224)
        After patchify: (BS, 56, 56, 96)
        After stage1:   (BS, 28, 28, 192)
        After stage2:   (BS, 14, 14, 384)
        After stage3:   (BS, 7, 7, 768)
        After stage4:   (BS, 7, 7, 768)
        Output logits:  (BS, n_classes)
    """
    def __init__(self, img_size, patch_size, emb_dim, blocks,
                 nheads, M, n_classes, stochastic_endpoint):
        super().__init__()

        H = img_size // patch_size
        W = img_size // patch_size
        dims = [emb_dim, 2*emb_dim, 4*emb_dim, 8*emb_dim]

        self.patchify = Patchify(3, emb_dim, patch_size)

        # Linearly increase across all blocks (inclusive endpoints)
        stoch_depth = list(np.linspace(0, stochastic_endpoint, sum(blocks)))
        
        # 4 Stages with stochastic depth aligned with the number of blocks in each stage.
        ind = 0
        self.stage1 = Stage(
            dims[0], H, W, M[0], 
            blocks[0], nheads[0], 
            stoch_depth[ind:ind+blocks[0]], 
            patch_merging=True, ratio=4.0)
        ind += blocks[0]
        H //= 2
        W //= 2
        self.stage2 = Stage(
            dims[1], H, W, M[1], 
            blocks[1], nheads[1], 
            stoch_depth[ind:ind+blocks[1]], 
            patch_merging=True, ratio=4.0)
        ind += blocks[1]
        H //= 2
        W //= 2
        self.stage3 = Stage(
            dims[2], H, W, M[2], 
            blocks[2], nheads[2], 
            stoch_depth[ind:ind+blocks[2]], 
            patch_merging=True, ratio=4.0)
        ind += blocks[2]
        H //= 2
        W //= 2
        self.stage4 = Stage(
            dims[3], H, W, M[3], 
            blocks[3], nheads[3], 
            stoch_depth[ind:ind+blocks[3]], 
            patch_merging=False, ratio=4.0)

        self.norm = nn.LayerNorm(dims[3])
        self.head = nn.Linear(dims[3], n_classes)
    
    def forward(self, x):
        """
        Forward pass.

        Args:
            x: Input tensor of shape (BS, 3, img_size, img_size).

        Returns:
            Class logits of shape (BS, n_classes).
        """
        # x shape = (BS, 3, 224, 224)
        x = self.patchify(x) # (BS, H//patch_size, W//patch_size, embed_dim)
        x = self.stage1(x)
        x = self.stage2(x)
        x = self.stage3(x)
        x = self.stage4(x)
        x = self.norm(x).mean(dim=(1, 2)) # Global average pooling over H, W.
        return self.head(x)

In [ ]:
from torch.optim.lr_scheduler import SequentialLR, LinearLR, CosineAnnealingLR
import json

model = SwinTransformer(
    img_size=64, patch_size=4, emb_dim=64,
    blocks=[2, 2, 4, 2], nheads=[2, 4, 8, 16],
    M=[8, 8, 4, 2], n_classes=200, stochastic_endpoint=0.1).to(device)

loss_fn = nn.CrossEntropyLoss()

# Scheduler Epochs
warmup_epochs = 5  # Gradually increase lr for the first 5 epochs
cosine_epochs = 95 # Cosine-anneal lr for the remaining 95 epochs

# AdamW optimizer, weight decay taken directly from the paper.
# fused=True uses a fused CUDA kernel
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=0.05, fused=True)

# Linear warmup scheduler. Start at 1% of lr and gradually fo up to 100%
warmup_scheduler = LinearLR(optimizer, start_factor=0.01, end_factor=1.0, total_iters=warmup_epochs)
# Cosine annealing scheduler. Decay lr following a cosine curve
cosine_scheduler = CosineAnnealingLR(optimizer, T_max=cosine_epochs)
# Run warmup_scheduler first, then cosine_scheduler
scheduler = SequentialLR(optimizer, schedulers=[warmup_scheduler, cosine_scheduler], milestones=[warmup_epochs])
# GradScaler for Automatic Mixed Precision (AMP). Saves VRAM.
scaler = torch.cuda.amp.GradScaler(enabled=(device.type=="cuda"))

best_val_acc = 0.0
history = {
    "train_loss": [],
    "train_acc": [],
    "val_loss": [],
    "val_acc": [],
    "lr": []
}

for epoch in range(warmup_epochs + cosine_epochs):
    model.train()
    total_loss = 0.0
    correct = 0
    total = 0
    
    for xb, yb in train_loader:
        xb = xb.to(device)
        yb = yb.to(device)
        optimizer.zero_grad(set_to_none=True) # set_to_none saves a bit of memory.
        # Forward and loss in mixed precision
        with torch.cuda.amp.autocast(enabled=(device.type=="cuda")):
            yhat = model(xb)
            loss = loss_fn(yhat, yb)
        # Backprop with scaled loss
        scaler.scale(loss).backward()
        # unscale before clipping, then clip
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        # Step optimizer only goes if gradients are finite
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item() * xb.size(0)
        preds = torch.argmax(yhat, dim=1)
        correct += (preds == yb).sum().item()
        total += xb.size(0)
    # Loss and accuracy
    avg_loss = total_loss / total
    accuracy = correct / total

    # Validation
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for xb, yb in val_loader:
            xb = xb.to(device)
            yb = yb.to(device)
            with torch.cuda.amp.autocast(enabled=(device.type == "cuda")):
                yhat = model(xb)
                loss = loss_fn(yhat, yb)
            val_loss += loss.item() * xb.size(0)
            preds = torch.argmax(yhat, dim=1)
            val_correct += (preds == yb).sum().item()
            val_total += xb.size(0)
    
    # Validation loss and accuracy
    avg_val_loss = val_loss / val_total
    val_accuracy = val_correct / val_total
    
    # Get current learning rate. I had this wrong before, I was grabbing the past lr instead of current
    current_lr = optimizer.param_groups[0]["lr"]

    history["train_loss"].append(avg_loss)
    history["train_acc"].append(accuracy)
    history["val_loss"].append(avg_val_loss)
    history["val_acc"].append(val_accuracy)
    history["lr"].append(current_lr)

    with open("training_history.json", "w") as f:
        json.dump(history, f)

    if val_accuracy > best_val_acc:
        best_val_acc = val_accuracy
        torch.save({
            "model": model.state_dict(),
            "optimizer": optimizer.state_dict(),
            "scheduler": scheduler.state_dict(),
            "epoch": epoch,
            "best_val_acc": best_val_acc
        }, "swin_t_best.pt")
        print("Saved new best model.")

    scheduler.step()

    print(f"Epoch {epoch+1}: "
          f"Train Loss: {avg_loss:.4f}, Train Acc: {accuracy:.4f}, "
          f"Val Loss: {avg_val_loss:.4f}, Val Acc: {val_accuracy:.4f}")

C:\Users\rowes\AppData\Local\Temp\ipykernel_18260\3172234026.py:26: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(device.type=="cuda"))
C:\Users\rowes\AppData\Local\Temp\ipykernel_18260\3172234026.py:48: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device.type=="cuda")):
C:\Users\rowes\AppData\Local\Temp\ipykernel_18260\3172234026.py:77: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device.type == "cuda")):


Saved new best model.
Epoch 1: Train Loss: 4.9961, Train Acc: 0.0305, Val Loss: 4.8224, Val Acc: 0.0481
Saved new best model.
Epoch 2: Train Loss: 4.4569, Train Acc: 0.0796, Val Loss: 3.9954, Val Acc: 0.1405
Saved new best model.
Epoch 3: Train Loss: 3.9401, Train Acc: 0.1400, Val Loss: 3.7650, Val Acc: 0.1715
Saved new best model.
Epoch 4: Train Loss: 3.6598, Train Acc: 0.1807, Val Loss: 3.5394, Val Acc: 0.2026
Saved new best model.
Epoch 5: Train Loss: 3.4769, Train Acc: 0.2119, Val Loss: 3.3565, Val Acc: 0.2365


c:\Users\rowes\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\optim\lr_scheduler.py:232: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


Saved new best model.
Epoch 6: Train Loss: 3.3371, Train Acc: 0.2327, Val Loss: 3.2954, Val Acc: 0.2392
Saved new best model.
Epoch 7: Train Loss: 3.1219, Train Acc: 0.2742, Val Loss: 3.1144, Val Acc: 0.2778
Saved new best model.
Epoch 8: Train Loss: 2.9474, Train Acc: 0.3052, Val Loss: 3.1205, Val Acc: 0.2830
Saved new best model.
Epoch 9: Train Loss: 2.7992, Train Acc: 0.3318, Val Loss: 2.9446, Val Acc: 0.3118
Saved new best model.
Epoch 10: Train Loss: 2.6728, Train Acc: 0.3566, Val Loss: 2.9510, Val Acc: 0.3177
Epoch 11: Train Loss: 2.5493, Train Acc: 0.3807, Val Loss: 2.9413, Val Acc: 0.3158
Saved new best model.
Epoch 12: Train Loss: 2.4395, Train Acc: 0.3999, Val Loss: 2.9088, Val Acc: 0.3263
Saved new best model.
Epoch 13: Train Loss: 2.3367, Train Acc: 0.4202, Val Loss: 2.8796, Val Acc: 0.3342
Epoch 14: Train Loss: 2.2344, Train Acc: 0.4393, Val Loss: 2.9586, Val Acc: 0.3219
Epoch 15: Train Loss: 2.1385, Train Acc: 0.4591, Val Loss: 2.9390, Val Acc: 0.3280
Epoch 16: Train Loss

224 images by 224 was way too much for my GPU. So I went down to 64 x 64 and adjusted. I didn't want M to be 2 so I modified so each stage has a different M. Entire architecture was adapted and controlled.